## Part 4. Human in the Loop 실습 - 이메일 회신 챗봇

목표
- 고객사로부터 받은 이메일에 대한 회신을 작성하는 챗봇을 구현합니다.
- 챗봇이 작성한 초안을 사람이 검토하고 승인/수정할 수 있는 Human-in-the-Loop 시스템을 구축합니다.
- interrupt와 Command를 활용하여 실행을 제어합니다.

시나리오
- 고객사(TechCorp)로부터 제품 문의 이메일이 도착했습니다.
- 챗봇이 자동으로 회신 초안을 작성합니다.
- 담당자가 초안을 검토하여 승인 또는 수정 요청을 합니다.
- 최종 승인된 이메일이 발송됩니다.

샘플 이메일
```
보내는 사람: kim@techcorp.com
제목: AI 솔루션 도입 문의

안녕하세요,

저희 회사에서 고객 서비스 자동화를 위한 AI 솔루션 도입을 검토하고 있습니다.
귀사의 LangGraph 기반 솔루션에 대해 다음 사항을 문의드립니다:

1. 기업용 라이선스 가격
2. 기술 지원 범위
3. 커스터마이징 가능 여부
4. 도입 사례

빠른 회신 부탁드립니다.

감사합니다.
김철수 부장
TechCorp
```

In [ ]:
from dotenv import load_dotenv
from langchain_teddynote import logging

load_dotenv(override=True)

# 프로젝트 이름
logging.langsmith("LangGraph-Exercises")

In [ ]:
# 준비 코드
from typing import Annotated, Literal
from typing_extensions import TypedDict

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.types import Command, interrupt

### 1. State 정의 및 모델 설정

- 이메일 회신 챗봇을 위한 State를 정의하세요.
- State에는 messages, email_draft, approval_status가 포함되어야 합니다.
- ChatOpenAI 모델(gpt-4.1, temperature=0.3)을 생성하세요.

In [ ]:
# 실습 코드
# TODO: State를 정의하세요. messages, email_draft, approval_status, human_feedback 필드를 포함해야 합니다.

class EmailState(TypedDict):
    messages: # 코드 입력: add_messages 사용
    email_draft: # 코드 입력: str 타입
    approval_status: # 코드 입력: Literal["pending", "approved", "rejected"] 타입
    human_feedback: # 코드 입력: str 타입 (수정 요청 피드백 저장)

# TODO: LLM 모델을 정의하세요.
llm = # 코드 입력

In [ ]:
# 정답 코드
# TODO: State를 정의하세요. messages, email_draft, approval_status, human_feedback 필드를 포함해야 합니다.


class EmailState(TypedDict):
    messages: Annotated[list, add_messages]
    email_draft: str
    approval_status: Literal["pending", "approved", "rejected"]
    human_feedback: str  # 수정 요청 피드백 저장


# TODO: LLM 모델을 정의하세요.
llm = ChatOpenAI(model="gpt-4.1", temperature=0.3)

### 2. 이메일 초안 작성 노드

- 받은 이메일을 분석하고 회신 초안을 작성하는 노드를 구현하세요.
- 시스템 프롬프트를 활용하여 전문적이고 친근한 톤으로 작성하도록 지시하세요.
- email_draft와 approval_status를 "pending"으로 설정하여 반환하세요.

In [ ]:
# 실습 코드
# TODO: 이메일 초안을 작성하는 노드를 구현하세요.

def draft_email(state: EmailState):
    """받은 이메일에 대한 회신 초안을 작성합니다."""
    
    # 시스템 프롬프트 설정
    system_prompt = # 코드 입력: 이메일 회신 작성 지시
    
    # human_feedback이 있으면 프롬프트에 추가
    if state.get("human_feedback"):
        system_prompt += # 코드 입력: feedback 반영 지시 추가
    
    # 메시지 구성
    messages = # 코드 입력: SystemMessage와 state의 messages 결합
    
    # LLM으로 초안 생성
    response = # 코드 입력
    
    return {
        # 코드 입력: email_draft와 approval_status 반환
    }

In [ ]:
# 정답 코드
# TODO: 이메일 초안을 작성하는 노드를 구현하세요.


def draft_email(state: EmailState):
    """받은 이메일에 대한 회신 초안을 작성합니다."""

    # 시스템 프롬프트 설정
    system_prompt = """당신은 전문적인 비즈니스 이메일 작성 도우미입니다.
    고객의 문의에 대해 친절하고 전문적으로 답변하세요.
    구체적인 정보를 제공하되, 과도한 약속은 피하세요.
    회신 이메일만 작성하고, 다른 설명은 포함하지 마세요."""

    # human_feedback이 있으면 프롬프트에 추가
    if state.get("human_feedback"):
        system_prompt += f"\n\n[중요] 다음 피드백을 반영하여 이메일을 재작성하세요: {state['human_feedback']}"

    # 메시지 구성
    messages = [SystemMessage(content=system_prompt)] + state["messages"]

    # LLM으로 초안 생성
    response = llm.invoke(messages)

    return {"email_draft": response.content, "approval_status": "pending"}

### 3. Human Review 노드 구현

- interrupt를 사용하여 사람의 검토를 요청하는 노드를 구현하세요.
- 초안을 보여주고 승인/수정 요청을 받아 처리하세요.
- Command 객체로 받은 피드백을 처리하여 상태를 업데이트하세요.

In [ ]:
# 실습 코드
# TODO: Human Review 노드를 구현하세요.

def human_review(state: EmailState):
    """사람의 검토를 요청하고 피드백을 받습니다."""
    
    print("\n===== 이메일 초안 검토 요청 =====")
    print(f"\n{state['email_draft']}")
    print("\n================================")
    print("옵션: approve(승인) / reject(수정요청)")
    
    # interrupt를 호출하여 사람의 입력 대기
    human_input = # 코드 입력: interrupt 호출
    
    # 피드백 처리
    if human_input.get("action") == "approve":
        return # 코드 입력: approval_status를 "approved"로 설정
    else:
        # 수정 요청시 피드백을 human_feedback에 저장
        feedback = # 코드 입력: human_input에서 feedback 가져오기
        return # 코드 입력: human_feedback과 approval_status를 "rejected"로 설정

In [ ]:
# 정답 코드
# TODO: Human Review 노드를 구현하세요.


def human_review(state: EmailState):
    """사람의 검토를 요청하고 피드백을 받습니다."""

    print("\n===== 이메일 초안 검토 요청 =====")
    print(f"\n{state['email_draft']}")
    print("\n================================")
    print("옵션: approve(승인) / reject(수정요청)")

    # interrupt를 호출하여 사람의 입력 대기
    human_input = interrupt({"draft": state["email_draft"]})

    # 피드백 처리
    if human_input.get("action") == "approve":
        return {"approval_status": "approved"}
    else:
        # 수정 요청시 피드백을 human_feedback에 저장
        feedback = human_input.get("feedback", "다시 작성해주세요.")
        return {"human_feedback": feedback, "approval_status": "rejected"}

### 4. 조건부 라우팅 구현

- approval_status에 따라 다음 노드를 결정하는 라우팅 함수를 구현하세요.
- pending: human_review 노드로
- approved: send_email 노드로
- rejected: draft_email 노드로 (재작성)

In [ ]:
# 실습 코드
# TODO: 조건부 라우팅 함수를 구현하세요.

def route_approval(state: EmailState) -> str:
    """승인 상태에 따라 다음 노드를 결정합니다."""
    status = # 코드 입력: state에서 approval_status 가져오기
    
    if status == "pending":
        return # 코드 입력: "human_review" 반환
    elif status == "approved":
        return # 코드 입력: "send_email" 반환
    else:  # rejected
        return # 코드 입력: "draft_email" 반환

# TODO: 이메일 발송 노드를 구현하세요.
def send_email(state: EmailState):
    """최종 승인된 이메일을 발송합니다."""
    print("\n✅ 이메일이 발송되었습니다!")
    print(f"\n{state['email_draft']}")
    return # 코드 입력: AIMessage로 발송 완료 메시지 반환

In [ ]:
# 정답 코드
# TODO: 조건부 라우팅 함수를 구현하세요.


def route_approval(state: EmailState) -> str:
    """승인 상태에 따라 다음 노드를 결정합니다."""
    status = state.get("approval_status", "pending")

    if status == "pending":
        return "human_review"
    elif status == "approved":
        return "send_email"
    else:  # rejected
        return "draft_email"


# TODO: 이메일 발송 노드를 구현하세요.
def send_email(state: EmailState):
    """최종 승인된 이메일을 발송합니다."""
    print("\n✅ 이메일이 발송되었습니다!")
    print(f"\n{state['email_draft']}")
    return {"messages": [AIMessage(content="이메일이 성공적으로 발송되었습니다.")]}

### 5. 완전한 그래프 구성 및 실행

- 모든 노드를 연결하여 완전한 그래프를 구성하세요.
- checkpointer를 설정하여 상태를 저장할 수 있도록 하세요.
- 샘플 이메일로 그래프를 실행하고 interrupt와 Command를 사용하여 상호작용하세요.

In [ ]:
# 실습 코드
# TODO: 완전한 그래프를 구성하세요.

# 그래프 빌더 생성
builder = # 코드 입력: StateGraph 생성

# 노드 추가
builder.# 코드 입력: draft_email 노드 추가
builder.# 코드 입력: human_review 노드 추가
builder.# 코드 입력: send_email 노드 추가

# 엣지 추가
builder.# 코드 입력: START → draft_email
builder.# 코드 입력: draft_email → route_approval (조건부)
builder.# 코드 입력: human_review → route_approval (조건부)
builder.# 코드 입력: send_email → END

# 체크포인터 설정 및 컴파일
memory = # 코드 입력: InMemorySaver 생성
app = builder.# 코드 입력: checkpointer와 함께 컴파일

In [ ]:
# 정답 코드
# TODO: 완전한 그래프를 구성하세요.

# 그래프 빌더 생성
builder = StateGraph(EmailState)

# 노드 추가
builder.add_node("draft_email", draft_email)
builder.add_node("human_review", human_review)
builder.add_node("send_email", send_email)

# 엣지 추가
builder.add_edge(START, "draft_email")
builder.add_conditional_edges("draft_email", route_approval)
builder.add_conditional_edges("human_review", route_approval)
builder.add_edge("send_email", END)

# 체크포인터 설정 및 컴파일
memory = InMemorySaver()
app = builder.compile(checkpointer=memory)

### 6. 그래프 시각화 및 실행

- 그래프를 시각화하고 샘플 이메일로 실행하세요.
- interrupt 시점에서 Command를 사용하여 승인 또는 수정 요청을 보내세요.

In [ ]:
# 실습 코드
# TODO: 그래프를 시각화하세요.
from langchain_teddynote.graphs import visualize_graph

# 코드 입력: 그래프 시각화

In [ ]:
# 정답 코드
# TODO: 그래프를 시각화하세요.
from langchain_teddynote.graphs import visualize_graph

visualize_graph(app)

### 7. 테스트 및 결과 확인

In [ ]:
# 실습 코드
# TODO: 샘플 이메일로 그래프를 실행하세요.
from langchain.core.runnables import RunnableConfig

# 샘플 이메일
customer_email = """보내는 사람: kim@techcorp.com
제목: AI 솔루션 도입 문의

안녕하세요,

저희 회사에서 고객 서비스 자동화를 위한 AI 솔루션 도입을 검토하고 있습니다.
귀사의 LangGraph 기반 솔루션에 대해 다음 사항을 문의드립니다:

1. 기업용 라이선스 가격
2. 기술 지원 범위
3. 커스터마이징 가능 여부
4. 도입 사례

빠른 회신 부탁드립니다.

감사합니다.
김철수 부장
TechCorp"""

# 설정
config = # 코드 입력: RunnableConfig 생성 (thread_id 포함)

# 그래프 실행 (interrupt에서 중단됨)
result = # 코드 입력: app.invoke 실행

In [ ]:
# 정답 코드
# TODO: 샘플 이메일로 그래프를 실행하세요.
from langchain_core.runnables import RunnableConfig

# 샘플 이메일
customer_email = """보내는 사람: kim@techcorp.com
제목: AI 솔루션 도입 문의

안녕하세요,

저희 회사에서 고객 서비스 자동화를 위한 AI 솔루션 도입을 검토하고 있습니다.
귀사의 LangGraph 기반 솔루션에 대해 다음 사항을 문의드립니다:

1. 기업용 라이선스 가격
2. 기술 지원 범위
3. 커스터마이징 가능 여부
4. 도입 사례

빠른 회신 부탁드립니다.

감사합니다.
김철수 부장
TechCorp"""

# 설정
config = RunnableConfig(configurable={"thread_id": "email_001"})

# 그래프 실행 (interrupt에서 중단됨)
result = app.invoke({"messages": [HumanMessage(content=customer_email)]}, config=config)

approve 나 reject 명령어를 사용하여 승인하거나 재작성을 요청하세요.

In [ ]:
# 실습 코드
# TODO: Command를 사용하여 초안을 승인하거나 수정 요청하세요.

# 상태 확인
snapshot = # 코드 입력: app.get_state로 현재 상태 확인
print(f"현재 노드: {snapshot.next}")

# 승인하는 경우
app.invoke(
    # 코드 입력: Command 객체로 승인 액션 전달
)

# 또는 수정 요청하는 경우
# app.invoke(
#     # 코드 입력: Command 객체로 수정 요청 전달
# )

In [ ]:
# 정답 코드
# TODO: Command를 사용하여 초안을 승인하거나 수정 요청하세요.

# 상태 확인
snapshot = app.get_state(config)
print(f"현재 노드: {snapshot.next}")

# 승인하는 경우
# app.invoke(
#     Command(
#         resume={"action": "approve"},
#     ),
#     config=config,
# )

# 또는 수정 요청하는 경우
app.invoke(
    Command(
        resume={"action": "reject", "feedback": "emoji 를 잔뜩 써주세요."},
    ),
    config=config,
)